In [11]:
!mv pipelines/abalone pipelines/customer_churn

In [2]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt ./
import boto3
import os
import sagemaker
prefix = 'sagemaker/DEMO-xgboost-churn'
region = boto3.Session().region_name
default_bucket = sagemaker.session.Session().default_bucket()
RawData = boto3.Session().resource('s3')\
.Bucket(default_bucket).Object(os.path.join(prefix, 'data/RawData.csv'))\
.upload_file('./churn.txt')
print(os.path.join("s3://",default_bucket, prefix, 'data/RawData.csv'))


download: s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt to ./churn.txt
s3://sagemaker-us-east-1-272720691618/sagemaker/DEMO-xgboost-churn/data/RawData.csv


In [8]:
!curl https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-immersion-day/master/ML%20Pipelines%20scripts/preprocess.py > pipelines/customer_churn/preprocess.py
!curl https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-immersion-day/master/ML%20Pipelines%20scripts/pipeline.py > pipelines/customer_churn/pipeline.py
!curl https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-immersion-day/master/ML%20Pipelines%20scripts/evaluate.py > pipelines/customer_churn/evaluate.py


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2726  100  2726    0     0  29956      0 --:--:-- --:--:-- --:--:-- 29956
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9919  100  9919    0     0   569k      0 --:--:-- --:--:-- --:--:--  538k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2694  100  2694    0     0  26411      0 --:--:-- --:--:-- --:--:-- 26411


CONTRIBUTING.md  codebuild-buildspec.yml	    setup.py
LICENSE		 img				    tests
README.md	 pipelines			    tox.ini
Untitled.ipynb	 sagemaker-pipelines-project.ipynb
churn.txt	 setup.cfg
